# Packing Data Into Provided Sample Zarr

The point of this notebook is to prototype the packaging of the simulation data I generated via Blender into a dataset Zarr tree like what Cheng provided on his GitHub. This way, we can easily integrate the simulation into the existing data structure for training the tracking extension.

In [1]:
import os
import sys
from pathlib import Path
import pickle

import zarr
import numpy as np
import skimage.io
import pandas as pd
from numcodecs import Blosc

# Do some ugly path manipulation to find all packages
sys.path.append("../")
sys.path.append("../../")
from simulation.io_util.image_io_util import read_uviz
from simulation.common.projection import ray_length_to_zbuffer, zbuffer_to_pcloud
from simulation.common.geometry_util import (barycentric_interpolation, get_aabb, get_union_aabb)
from simulation.cloth_3d_util.util import quads2tris, axis_angle_to_matrix
from simulation.common.igl_util import query_uv_barycentric
from simulation.data_packager.cloth_3d_canonical_accessor import Cloth3DCanonicalAccessor

# Trying to avoid this import as it will execute the Zarr packaging code as is.
# from simulation.data_packager.smpl_cloth_zarr_v5_cheng import Cloth3DCanonicalAccessor

%load_ext autoreload
%autoreload 2


In [2]:
FILE_ROOT = Path(os.getcwd())
print("File root:", FILE_ROOT)
GARMENTNETS_ROOT = (FILE_ROOT / ".." / "..").absolute().resolve()
print("GarmentNets root:", GARMENTNETS_ROOT)
SIM_DATASET_DIR = (FILE_ROOT / ".." / "script_output" / "full_dataset_attempt_2").absolute()
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))

File root: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/data_packager
GarmentNets root: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking


In [3]:
# Convenience function for debugging
def print_nested_dict_types(d: dict):
    keys_and_values = [(k, v, 0) for k, v in d.items()][::-1]
    while len(keys_and_values) > 0:
        key, val, level = keys_and_values.pop(-1)
        # if
        tabs = '\t' * level
        if isinstance(val, dict):
            print(tabs + key)
            for subkey, subval in val.items():
                keys_and_values.append((subkey, subval, level + 1))
        else:
            print(tabs + key, type(val))

## TODOs:

- I should read the Blend files so that I can calculate exactly what the change in position/velocity of the "gripper" is at each timestep.
- Define interface for new Zarr 

### New Zarr Interface

refer to `zarr_structure_plan.md` for new planned structure.   


## Setup Output Zarr

- Use `<zarr_node>.create_group('group_name')` to create a group at the current node (group)
- Use `<var> = <group>.array(name='name', data=<your_numpy_array>)` to add an array to a group.

In [4]:
ZARR_FILEPATH = GARMENTNETS_ROOT / "data" / "garmentnets_tracking_simulation_dataset.zarr"

# r+ is read/write but the data must exist first.
# mode = 'r'  # Use for debugging first.
# mode = 'r+'
# zr = zarr.open(ZARR_FILEPATH.as_posix(), mode=mode)
compressor = Blosc(cname='zstd', clevel=6, shuffle=Blosc.BITSHUFFLE)

categories = ["Tshirt"]
rows = dict()
for category in categories:
    store = zarr.DirectoryStore(ZARR_FILEPATH / category)
    root = zarr.group(store=store, overwrite=False)
    sample_root = root.require_group('samples', overwrite=False)
    summary_root = root.require_group('summary', overwrite=False)
    rows[category] = {
        'compressor': compressor,
        'store': store,
        'root': root,
        'sample_root': sample_root,
        'summary_root': summary_root
    }
datasets_df = pd.DataFrame(list(rows.values()), index=rows.keys())

In [5]:
datasets_df

,compressor,store,root,sample_root,summary_root
Tshirt,"Blosc(cname='zstd', clevel=6, shuffle=BITSHUFF...","[.zgroup, summary/.zgroup, samples/.zgroup]","[samples, summary]",[],[]


In [6]:
type(root)

zarr.hierarchy.Group

## Prototyping Dynamics Zarr Functions

In [9]:
dynamics_seq_dir = SIM_DATASET_DIR / "00380_Tshirt_509" / "dynamics_seq_0"

In [10]:
# Load the simulation results for this dynamics run
results_filepath = dynamics_seq_dir / "dynamics_sim_results.pkl"
results_dict = pickle.load(results_filepath.open('rb'))

In [12]:
# Load the metadata
meta_path = dynamics_seq_dir.joinpath('meta.pk')
meta = pickle.load(meta_path.open('rb'))

In [15]:
# Tentative: load the canonical data? Or maybe pass it in if we need it to get canonical
# coordinates corresponding to the points in the point cloud of the dynamics sequence.

# Load in the images.
uviz_fpaths = list(dynamics_seq_dir.glob("*.exr"))
rgb_fpaths = list(dynamics_seq_dir.glob("*.png"))
assert (len(uviz_fpaths) == len(rgb_fpaths))
rows = list()
for i, (uviz_fpath, rgb_fpath) in enumerate(zip(uviz_fpaths, rgb_fpaths)):
    print(f"\tLoading rgb and uviz for frame {i}")
    uviz_dict = read_uviz(str(uviz_fpath.absolute()), index_dtype=np.uint8)
    rgb = skimage.io.imread(str(rgb_fpath.absolute()))

    row = uviz_dict
    row['rgb'] = rgb
    rows.append(row)

    # Only load up to the images that we have animations for.
    if i == results_dict["simulation_info"]["frame_end"]:
        break

images_df = pd.DataFrame(rows)

	Loading rgb and uviz for frame 0
	Loading rgb and uviz for frame 1
	Loading rgb and uviz for frame 2
	Loading rgb and uviz for frame 3
	Loading rgb and uviz for frame 4
	Loading rgb and uviz for frame 5
	Loading rgb and uviz for frame 6
	Loading rgb and uviz for frame 7
	Loading rgb and uviz for frame 8
	Loading rgb and uviz for frame 9
	Loading rgb and uviz for frame 10
	Loading rgb and uviz for frame 11
	Loading rgb and uviz for frame 12
	Loading rgb and uviz for frame 13
	Loading rgb and uviz for frame 14
	Loading rgb and uviz for frame 15
	Loading rgb and uviz for frame 16
	Loading rgb and uviz for frame 17
	Loading rgb and uviz for frame 18
	Loading rgb and uviz for frame 19
	Loading rgb and uviz for frame 20
	Loading rgb and uviz for frame 21
	Loading rgb and uviz for frame 22
	Loading rgb and uviz for frame 23
	Loading rgb and uviz for frame 24
	Loading rgb and uviz for frame 25
	Loading rgb and uviz for frame 26
	Loading rgb and uviz for frame 27
	Loading rgb and uviz for fram

In [21]:
# Reformat the images.
intrinsic = meta['camera']['intrinsic']
extrinsic_arr = np.array(meta['camera']['extrinsic_list'])[0]

In [23]:
rgb_arr = np.array(list(images_df.rgb))
uv_arr = np.array(list(images_df.uv))
index_arr = np.array(list(images_df.object_index))
# should specify index in meta
mask_arr = (index_arr == 1).squeeze()
# convert Cycles ray length to CV depth
# depth_arr = np.array(list(images_df.depth.apply(
#     lambda x: ray_length_to_zbuffer(x, intrinsic))))
# in Blender 2.90, Cycle's definition of depth changed to CV depth
depth_arr = np.array(list(images_df.depth))

In [27]:
print("rgb shape", rgb_arr.shape)
print("uv_arr shape", uv_arr.shape)
print("index_arr shape", index_arr.shape)
print("mask_arr shape", mask_arr.shape)
print("depth_arr shape", depth_arr.shape)




rgb shape (153, 1024, 1024, 3)
uv_arr shape (153, 1024, 1024, 2)
index_arr shape (153, 1024, 1024, 1)
mask_arr shape (153, 1024, 1024)
depth_arr shape (153, 1024, 1024, 1)


In [ ]:
# Generate point cloud in the global frame.
# NOTE: Have to translate by the negative of the amount the camera was translated for dynamics
# sequence recording.
point_cloud_arr = np.empty(rgb_arr.shape, dtype=np.float16)
assert(depth_arr.shape[0] == rgb_arr.shape[0])
for i in range(len(depth_arr)):
    depth = depth_arr[i]
    extrinsic = extrinsic_arr[i]
    pc_local = zbuffer_to_pcloud(depth, intrinsic)
    tx_world_camera = np.linalg.inv(extrinsic)
    pc_global = pc_local @ tx_world_camera[:3,:3].T + tx_world_camera[:3, 3]
    point_cloud_arr[i] = pc_global

## The Following Should Be Converted To A Function To Add A Single Sample's Simulations To Zarr

This was taken from `smpl_cloth_zarr_v5_cheng.convert_experiment()`

In [34]:
# Setup some debug variables that will eventually be used as function arguments.
sample_dir = SIM_DATASET_DIR / "00380_Tshirt_509"

# zarr configuration
sample_root = sample_root  # Probably the "Group" at which this sample is located.
compressor = compressor
accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)

In [35]:
# load metadata
meta_path = sample_dir.joinpath('meta.pk')
sim_result_path = sample_dir.joinpath('hanging_rest_state_results.pkl')
meta = pickle.load(meta_path.open('rb'))
sim_result = pickle.load(sim_result_path.open('rb'))

In [36]:
print_nested_dict_types(meta)

camera
	extrinsic_list <class 'list'>
	intrinsic <class 'numpy.ndarray'>
images
	rgb <class 'list'>
	uviz <class 'list'>
meta
	fabric <class 'str'>
	gender <class 'int'>
	garment_name <class 'str'>
	sample_id <class 'str'>


In [37]:
# check and read metadata
cloth_verts = sim_result['cloth_state']['verts']
cloth_faces = np.array(sim_result['cloth_state']['faces'])
# CLOTH3D data has all quad faces
assert(cloth_faces.shape[1] == 4)
cloth_uv_verts = sim_result['cloth_state']['uv_verts']
cloth_uv_faces = np.array(sim_result['cloth_state']['uv_faces'])
assert(cloth_uv_faces.shape[1] == 4)
grip_vertex_idx = sim_result['grip_vertex_idx']
assert(0 <= grip_vertex_idx < len(cloth_verts))

In [38]:
# load canonical data
canonical_data = accessor.get_sample_data(
    sample_id=meta['meta']['sample_id'],
    garment_name=meta['meta']['garment_name'])

In [39]:
# compute per-cloth-vertex nearest human vertex
human_verts = canonical_data['human_verts']
human_faces = canonical_data['human_faces']
cloth_canonical_verts = canonical_data['garment_verts']
cloth_texture = canonical_data['garment_texture']

In [40]:
# load images
# TODO: This will have to be heavily modified to work with the dynamics simulations.
uviz_fnames = meta['images']['uviz']
rgb_fnames = meta['images']['rgb']
assert(len(uviz_fnames) == len(rgb_fnames))
rows = list()
for uviz_fname, rgb_fname in zip(uviz_fnames, rgb_fnames):
    uviz_path = sample_dir.joinpath(uviz_fname)
    rgb_path = sample_dir.joinpath(rgb_fname)
    uviz_dict = read_uviz(str(uviz_path.absolute()), index_dtype=np.uint8)
    rgb = skimage.io.imread(str(rgb_path.absolute()))

    row = uviz_dict
    row['rgb'] = rgb
    rows.append(row)
images_df = pd.DataFrame(rows)

In [41]:
# reformat images
intrinsic = meta['camera']['intrinsic']
extrinsic_arr = np.array(list(meta['camera']['extrinsic_list']))
rgb_arr = np.array(list(images_df.rgb))
uv_arr = np.array(list(images_df.uv))
index_arr = np.array(list(images_df.object_index))
# should specify index in meta
mask_arr = (index_arr == 1).squeeze()
# convert Cycles ray length to CV depth
# depth_arr = np.array(list(images_df.depth.apply(
#     lambda x: ray_length_to_zbuffer(x, intrinsic))))
# in Blender 2.90, Cycle's definition of depth changed to CV depth
depth_arr = np.array(list(images_df.depth))

In [42]:
# generate point cloud in global frame
point_cloud_arr = np.empty(rgb_arr.shape, dtype=np.float16)
assert(len(depth_arr) == len(extrinsic_arr))
for i in range(len(depth_arr)):
    depth = depth_arr[i]
    extrinsic = extrinsic_arr[i]
    pc_local = zbuffer_to_pcloud(depth, intrinsic)
    tx_world_camera = np.linalg.inv(extrinsic)
    pc_global = pc_local @ tx_world_camera[:3,:3].T + tx_world_camera[:3, 3]
    point_cloud_arr[i] = pc_global

In [43]:
# extract cloth point cloud
pc_points = point_cloud_arr[mask_arr]
pc_uv = uv_arr[mask_arr]
pc_rgb = rgb_arr[mask_arr]
pc_sizes = np.sum(mask_arr, (1,2))
assert(np.sum(pc_sizes) == len(pc_points))

In [44]:
# compute canonical coordinate for point cloud
cloth_uv_faces_tri = quads2tris(cloth_uv_faces)
cloth_faces_tri = quads2tris(cloth_faces)

query_uv = pc_uv
target_uv_verts = cloth_uv_verts
target_uv_faces = cloth_uv_faces_tri

barycentric, proj_face_idx = query_uv_barycentric(pc_uv, cloth_uv_verts, cloth_uv_faces_tri)
pc_canonical = barycentric_interpolation(
    barycentric, cloth_canonical_verts, cloth_faces_tri[proj_face_idx])

In [45]:
# compute human/cloth aabb
human_aabb = get_aabb(human_verts)
cloth_aabb = get_aabb(cloth_canonical_verts)
aabb = get_union_aabb(human_aabb, cloth_aabb)

In [46]:
# write to zarr
experiment_group = sample_root.require_group(sample_dir.stem, overwrite=False)
image_group = experiment_group.require_group('image', overwrite=True)
point_cloud_group = experiment_group.require_group('point_cloud', overwrite=True)
misc_group = experiment_group.require_group('misc', overwrite=True)

In [47]:
# write misc arrays
misc_data = {
    'cloth_verts': cloth_verts.astype(np.float32),
    'cloth_faces': cloth_faces.astype(np.uint16),
    'cloth_uv_verts': cloth_uv_verts.astype(np.float32),
    'cloth_uv_faces': cloth_uv_faces.astype(np.uint16),
    'cloth_canonical_verts': cloth_canonical_verts.astype(np.float32),
    'human_verts': human_verts.astype(np.float32),
    'human_faces': human_faces.astype(np.uint16),
    'cloth_aabb': cloth_aabb.astype(np.float32),
    'human_aabb': human_aabb.astype(np.float32),
    'intrinsic': intrinsic.astype(np.float32),
    'extrinsic_list': extrinsic_arr.astype(np.float32),
    'cloth_texture': cloth_texture.astype(np.uint8)
}
for key, data in misc_data.items():
    misc_group.array(
        name=key, data=data, chunks=data.shape,
        compressor=compressor, overwrite=True)

In [48]:
# write image arrays
image_data = {
    'rgb': rgb_arr.astype(np.uint8),
    'uv': uv_arr.astype(np.float16),
    'depth': depth_arr.astype(np.float16),
    'mask': np.expand_dims(mask_arr, axis=-1)
}
for key, data in image_data.items():
    image_group.array(
        name=key, data=data, chunks=(1,) + data.shape[1:],
        compressor=compressor, overwrite=True)

In [49]:
# write point cloud arrays
pc_data = {
    'point': pc_points.astype(np.float16),
    'uv': pc_uv.astype(np.float16),
    'rgb': pc_rgb.astype(np.uint8),
    'canonical_point': pc_canonical.astype(np.float16),
    'sizes': pc_sizes.astype(np.int64)
}
for key, data in pc_data.items():
    point_cloud_group.array(
        name=key, data=data, chunks=data.shape,
        compressor=compressor, overwrite=True)

In [50]:
# set attrs
meta_attr = meta['meta']
attrs = {
    'sample_id': meta_attr['sample_id'],
    'garment_name': meta_attr['garment_name'],
    'gender': meta_attr['gender'],
    'fabric': meta_attr['fabric'],
    'grip_vertex_idx': grip_vertex_idx
}
experiment_group.attrs.put(attrs)

In [ ]:
# Add some garbage so that when I hit "run all" it errors on this cell and halts execution.
asdfasdfasdfasd

### Get Simulation Images

Image arrays:
- `depth`
- `mask`
- `rgb`
- `uv`

### Get Simulation Point Clouds

These point clouds will be generated by using the camera intrinsic, the renders, and the masks from the renders. 

When saving the point clouds to the Zarr, the point clouds should be translated by the Z offset used to offset the camera. This was done so as to make sure the Tshirt fit in the camera frame across the full dynamics simulation (hopefully).

Need the following from the simulation data:
- `canonical_point`
- `point`
- `rgb`
- `sizes`
- `uv`

In [28]:
zr['Tshirt/samples/00380_Tshirt_509/image'].info

Name,/Tshirt/samples/00380_Tshirt_509/image
Type,zarr.hierarchy.Group
Read-only,True
Store type,zarr.storage.DirectoryStore
No. members,4
No. arrays,4
No. groups,0
Arrays,"depth, mask, rgb, uv"


### Tentative: Add Miscellaneous Info?

Misc. Arrays:
- `cloth_aabb`
- `cloth_canonical_aabb`
- `cloth_canonical_verts`
- `cloth_faces`
- `cloth_texture`
- `cloth_uv_faces`
- `cloth_uv_verts`
- `cloth_verts`
- `extrinsic_list`
- `human_canonical_aabb`
- `human_faces`
- `human_verts`
- `intrinsic`


In [27]:
zr['Tshirt/samples/00380_Tshirt_509/misc'].info

Name,/Tshirt/samples/00380_Tshirt_509/misc
Type,zarr.hierarchy.Group
Read-only,True
Store type,zarr.storage.DirectoryStore
No. members,13
No. arrays,13
No. groups,0
Arrays,"cloth_aabb, cloth_canonical_aabb, cloth_canonical_verts, cloth_faces, cloth_texture, cloth_uv_faces, cloth_uv_verts, cloth_verts, extrinsic_list, human_canonical_aabb, human_faces, human_verts, intrinsic"
